In [192]:
import pandas as pd
import torch
import numpy as np
from datetime import datetime
import numpy as np
import datetime
import matplotlib.pyplot as plt
from scipy.stats import normaltest
from sklearn.preprocessing import MinMaxScaler
import random
import copy
from sklearn.kernel_ridge import KernelRidge
import re
import nltk
from nltk import word_tokenize

In [193]:
# previously loaded data from 2013 - 2019 - now running on new data from 2005 to 2019
# data = pd.read_csv("../Current_Data/New_data_updated_names.csv", usecols=['Country','Date','Cases','Deaths',' 
#                   Summary'], parse_dates=["Date"])
data = pd.read_csv("../Current_Data/data2005to2019.csv", usecols=['Country','Date','Cases','Deaths','Summary'],  
                  parse_dates=["Date"])

data.head()


,Country,Date,Cases,Deaths,Summary
0,Benin,2005-01-03,13,8.0,Note that year 2004 had 53 weeks. Hence the fi...
1,Burkina Faso,2005-01-03,106,23.0,Note that year 2004 had 53 weeks. Hence the fi...
2,Ivory Coast,2005-01-03,5,0.0,Note that year 2004 had 53 weeks. Hence the fi...
3,Mali,2005-01-03,18,0.0,Note that year 2004 had 53 weeks. Hence the fi...
4,Niger,2005-01-03,29,4.0,Note that year 2004 had 53 weeks. Hence the fi...


In [194]:
#note chad is at the end of the list because, despite the list being alphabetically organized, in WHO reports
#chad was recorded as Tchad
country_list = ['Benin', 'Burkina Faso', 'Burundi', 'Cameroun', 'Centrafrique', "Ivory Coast", 'Ethiopia', 'Ghana',
               'Guinea', 'Guinea Bissau', 'Gambia', 'Kenya', 'Mali', 'Mauritania', 'Niger', 'Nigeria', 'Dem. Rep. Congo', 
                'Senegal' , 'South Sudan', 'Sudan', 'Sierra Lone', 'Tanzania', 'Chad', 'Togo', 'Uganda']
bordering_countries = {"Benin": ["Togo", "Nigeria", "Niger", "Burkina Faso"], 
                       "Burkina Faso": ["Mali", "Niger", "Benin" ,"Togo", "Ghana", "Ivory Coast"], 
                       "Burundi": ["Dem. Rep. Congo", "Tanzania"], 
                       "Cameroun": ["Centrafrique", "Nigeria", "Guinea", "Chad"], 
                       "Centrafrique": ["Cameroun", "Sudan", "South Sudan","Dem. Rep. Congo", "Chad"], 
                       "Ivory Coast": ["Ghana", "Mali", "Burkina Faso"], 
                       "Ethiopia": ["Sudan", "South Sudan", "Kenya"], 
                       "Ghana":["Guinea", "Ivory Coast", "Burkina Faso", "Togo"], 
                       "Guinea":["Ivory Coast", "Guinea Bissau", "Mali", "Senegal", "Sierra Lone"], 
                       "Guinea Bissau": ["Senegal", "Guinea"], 
                       "Gambia":["Senegal"], 
                       "Kenya": ["Ethiopia", "South Sudan", "Uganda"],
                       "Mali": ["Niger", "Burkina Faso", "Ivory Coast", "Guinea", "Mauritania", "Senegal"], 
                       "Mauritania":["Mali", "Senegal"], 
                       "Niger": ["Benin", "Nigeria", "Burkina Faso", "Mali", "Chad"], 
                       "Nigeria": ["Benin", "Cameroun", "Niger", "Chad"], 
                       "Dem. Rep. Congo": ["Burundi", "Centrafrique", "South Sudan", "Tanzania", "Uganda"], 
                       "Senegal":["Gambia", "Guinea", "Guinea Bissau", "Mali","Mauritania"], 
                       "South Sudan":["Sudan", "Ethiopia", "Kenya", "Dem. Rep. Congo", "Uganda","Centrafrique"], 
                       "Sudan":["Ethiopia", "South Sudan", "Centrafrique", "Chad"], 
                       "Sierra Lone": ["Guinea"], 
                       "Tanzania":["Kenya", "Uganda", "Burundi", "Dem. Rep. Congo"], 
                       "Chad":["Sudan", "Centrafrique", "Cameroun", "Niger", "Nigeria"], 
                       "Togo":["Ghana", "Burkina Faso","Benin", "Guinea"],
                       "Uganda":["Kenya", "South Sudan", "Dem. Rep. Congo", "Tanzania"]}
#this lambda function returns the index of the country from the corresponding country_list array
get_indices = lambda x: country_list.index(str(x).rstrip())
country_list.index("Guinea")

8

In [195]:
#creates a list of bordering countries for each datapoint with the countries represented as indices of country_list array
data_bordering_countries = pd.DataFrame(data["Country"].apply(lambda x: 
                                                              list(
                                                                  map(get_indices, 
                                                                      bordering_countries[str(x).rstrip()]))
                                                                   ))
data.insert(5, "Bordering Countries", data_bordering_countries)
#replace country names in country column with corresponding indices from country_list array
data["Country"] = data["Country"].apply(lambda x: country_list.index(str(x).rstrip()))
data.loc[0:20]

,Country,Date,Cases,Deaths,Summary,Bordering Countries
0,0,2005-01-03,13,8.0,Note that year 2004 had 53 weeks. Hence the fi...,"[23, 15, 14, 1]"
1,1,2005-01-03,106,23.0,Note that year 2004 had 53 weeks. Hence the fi...,"[12, 14, 0, 23, 7, 5]"
2,5,2005-01-03,5,0.0,Note that year 2004 had 53 weeks. Hence the fi...,"[7, 12, 1]"
3,12,2005-01-03,18,0.0,Note that year 2004 had 53 weeks. Hence the fi...,"[14, 1, 5, 8, 13, 17]"
4,14,2005-01-03,29,4.0,Note that year 2004 had 53 weeks. Hence the fi...,"[0, 15, 1, 12, 22]"
5,16,2005-01-03,61,12.0,Note that year 2004 had 53 weeks. Hence the fi...,"[2, 4, 18, 21, 24]"
6,22,2005-01-03,4,0.0,Note that year 2004 had 53 weeks. Hence the fi...,"[19, 4, 3, 14, 15]"
7,23,2005-01-03,7,1.0,Note that year 2004 had 53 weeks. Hence the fi...,"[7, 1, 0, 8]"
8,0,2005-01-10,7,1.0,"At week 2, 2005, the district of Saponé in Bur...","[23, 15, 14, 1]"
9,1,2005-01-10,156,37.0,"At week 2, 2005, the district of Saponé in Bur...","[12, 14, 0, 23, 7, 5]"


In [196]:
def find_next_batch(count):
    initialCount = count
    found = False
    while not found:
        count += 1
        if count == len(data) - 1 or data["Date"].loc[count + 1] != data["Date"].loc[count]:
            found = True
            break
    return data.iloc[initialCount:count+1]

In [197]:
find_next_batch(0)

,Country,Date,Cases,Deaths,Summary,Bordering Countries
0,0,2005-01-03,13,8.0,Note that year 2004 had 53 weeks. Hence the fi...,"[23, 15, 14, 1]"
1,1,2005-01-03,106,23.0,Note that year 2004 had 53 weeks. Hence the fi...,"[12, 14, 0, 23, 7, 5]"
2,5,2005-01-03,5,0.0,Note that year 2004 had 53 weeks. Hence the fi...,"[7, 12, 1]"
3,12,2005-01-03,18,0.0,Note that year 2004 had 53 weeks. Hence the fi...,"[14, 1, 5, 8, 13, 17]"
4,14,2005-01-03,29,4.0,Note that year 2004 had 53 weeks. Hence the fi...,"[0, 15, 1, 12, 22]"
5,16,2005-01-03,61,12.0,Note that year 2004 had 53 weeks. Hence the fi...,"[2, 4, 18, 21, 24]"
6,22,2005-01-03,4,0.0,Note that year 2004 had 53 weeks. Hence the fi...,"[19, 4, 3, 14, 15]"
7,23,2005-01-03,7,1.0,Note that year 2004 had 53 weeks. Hence the fi...,"[7, 1, 0, 8]"


In [198]:
find_next_batch(5057)

,Country,Date,Cases,Deaths,Summary,Bordering Countries
5057,0,2019-12-16,55,4.0,"From week 49 to 52 (December 2019), 8 district...","[23, 15, 14, 1]"
5058,1,2019-12-16,87,1.0,"From week 49 to 52 (December 2019), 8 district...","[12, 14, 0, 23, 7, 5]"
5059,2,2019-12-16,6,0.0,"From week 49 to 52 (December 2019), 8 district...","[16, 21]"
5060,4,2019-12-16,13,3.0,"From week 49 to 52 (December 2019), 8 district...","[3, 19, 18, 16, 22]"
5061,5,2019-12-16,9,0.0,"From week 49 to 52 (December 2019), 8 district...","[7, 12, 1]"
5062,7,2019-12-16,34,2.0,"From week 49 to 52 (December 2019), 8 district...","[8, 5, 1, 23]"
5063,8,2019-12-16,33,1.0,"From week 49 to 52 (December 2019), 8 district...","[5, 9, 12, 17, 20]"
5064,10,2019-12-16,0,0.0,"From week 49 to 52 (December 2019), 8 district...",[17]
5065,12,2019-12-16,44,0.0,"From week 49 to 52 (December 2019), 8 district...","[14, 1, 5, 8, 13, 17]"
5066,13,2019-12-16,0,0.0,"From week 49 to 52 (December 2019), 8 district...","[12, 17]"


In [199]:
data.loc[0:54]["Country"].unique()

array([ 0,  1,  5, 12, 14, 16, 22, 23,  7])

### Convert Weekly Data into Montly to Analyze Distributions

In [200]:
def update_batch(temp_batch, curr_batch):
    for index, row in curr_batch.iterrows():
            #check if the country is in the temp monthly batch, if not add it, otherwise update Cases, Deaths, and Summary
            if row["Country"] not in temp_batch["Country"].unique():
                if len(temp_batch) != 0:
                    row["Date"] = temp_batch["Date"].iloc[0]
                temp_batch = temp_batch.append(row)
            else:
                line_separator = "." if temp_batch.loc[temp_batch.Country == row["Country"], "Summary"].item()[-1] != "." else ""
                temp_batch.loc[temp_batch.Country == row["Country"], "Cases"] += row["Cases"]
                temp_batch.loc[temp_batch.Country == row["Country"], "Deaths"] += row["Deaths"]
                temp_batch.loc[temp_batch.Country == row["Country"], "Summary"] += line_separator + row["Summary"]
    return temp_batch
    
#Convert the weekly data into montly data (by summing over the Cases, Deaths, and concatenating the summaries)
count = 0
new_monthly_data = pd.DataFrame(columns=["Country", "Date", "Cases", "Deaths", "Summary", "Bordering Countries"])
#create a temporary dataframe for current month which will be concatenated to the final dataframe
temp_batch = new_monthly_data.copy()
curr_month = 1
#The upper-bound is determined by the number which when passed to the find_next_batch function will return the final batch
while count <= 5057:
    curr_batch = find_next_batch(count)
    count += len(curr_batch)
    #if the new batch is of a different month, add it to the new_monthly_data dataframe and empty the temp_batch for the new month
    if curr_batch["Date"].iloc[0].month != curr_month:
        curr_month = curr_batch["Date"].iloc[0].month
        new_monthly_data = pd.concat([new_monthly_data, temp_batch], ignore_index = True)
        temp_batch = pd.DataFrame(columns=["Country", "Date", "Cases", "Deaths", "Summary", "Bordering Countries"])
        #ensures last batch is added to the data
        if count >= 5057:
            new_monthly_data = pd.concat([new_monthly_data, find_next_batch(count)], ignore_index = True)
            break
    temp_batch = update_batch(temp_batch, curr_batch)

new_monthly_data.to_csv("../Current_Data/Month_data.csv", index=False)

    

### Plotting Histograms With No. Of Cases for Each Country

In [201]:
"""This function will go through any time-series data in a pandas dataframe e.g. weekly, monthly cases etc. and 
return two dic with 
data_dict: countries as keys and a list of datapoints (weekly, monthly, yearly etc.) as the values
threshold_dict: countries as keys and list representing the 30th and 70th percentiles as values

"""

#This is an update get_thresholds method after realizing that the function below computed independent thresholds
#for each country for each year => we weren't getting an even distribution across all states
#commented out the code which computes the thresholds in case we need to go back to it
def get_thresholds(df):
    threshold_dict = {}
    for country in list(df["Country"].unique()):
        country_df = df.loc[df["Country"] == country]
        first_percentile = country_df["Cases"].quantile(0.3)
        second_percentile = country_df["Cases"].quantile(0.7)
        threshold_dict[country] = [first_percentile, second_percentile]
    return threshold_dict


def time_series_to_dict(df):
    data_dict = {}
    threshold_dict = {}
    for i in range(2005, 2020):
        curr_yr = df.loc[df.Date.dt.year == i]
        countries_list=list(curr_yr["Country"].unique())
        for country in countries_list:
            country_curr_yr = curr_yr[curr_yr.Country == country]
#             first_percentile = country_curr_yr["Cases"].quantile(0.3)
#             second_percentile =  country_curr_yr["Cases"].quantile(0.7)
#             threshold_dict[country] = [first_percentile, second_percentile]
            if country not in data_dict:
                data_dict[country] = country_curr_yr["Cases"].tolist()
            else:
                data_dict[country]+= country_curr_yr["Cases"].tolist()
#     if thresholds_needed:
#         return data_dict, threshold_dict
    return data_dict

monthly_data_dict = time_series_to_dict(new_monthly_data)


#Apply a normal test to see if the data for each country reflects a Gaussian distribution.
# A 5% significiance level is used for each country (and the null hypothesis is rejected if p-value <0.05)

# for key, values in monthly_data_dict.items():
#     if len(values) > 20:

#         plt.hist(values)
#         plt.show()
#         stat, p = normaltest(values)
#         print(country_list[key])
#         print('Statistics=%.3f, p=%.3f' % (stat, p))
#         # interpret
#         alpha = 0.05
#         if p > alpha:
#             print('Sample looks Gaussian (fail to reject H0)')
#         else:
#             print('Sample does not look Gaussian (reject H0)')
        
        

### Determining State Thresholds for Each Country 
To determine the threshold for each country (i.e. no. of cases for each country that will determine whether the severity is low, medium, or high), we will start by computing independent thresholds for each country. Since each country has different populations, factors affecting the spread of the pandemic (e.g. no. of people vaccinated etc.), the same number of cases in different locations has different implications (e.g. 10 cases for a population of 100 is much more dangerous than 10 for 1,000,000. The thresholds for low, medium, and high will be the values that account fo 30%, 40%, and 30% respectively. In other words, each country will have two thresholds at the 30th and 70th and percentiles (calculated from the time_series_to_dict function above).

0 = low  

1 = medium 

2 = high

In [202]:
#what to do data with thresholds of 0,0? For now drop
thresholds_weekly_data = get_thresholds(data)
for country, thresholds in thresholds_weekly_data.items():
    #*NOTE updated thresholds which are now ignored to include 0 and 1 or 0 and 2
    if (thresholds[0] == 0 and thresholds[1] == 0) or (thresholds[0] == 0 and thresholds[1] <= 2):
        indexNames = data[data['Country'] == country ].index
        data = data.drop(indexNames, axis=0)
        print(country)

weekly_data_dict = time_series_to_dict(data)
thresholds_weekly_data = get_thresholds(data)

data["State"] = 0
def find_state(country_threshold_list, no_cases):
    if no_cases >= float(country_threshold_list[1]):
        return 2
    elif no_cases >= float(country_threshold_list[0]):
        return 1
    else:
        return 0


#find the state for each country by comparing the number of cases to the corresponding thresholds 
data["State"] = data.apply(lambda x: find_state(thresholds_weekly_data[x["Country"]], x["Cases"]) ,axis=1)
data.head()
# scaler = MinMaxScaler()
# scaled_data = data.copy()
# scaled_data[["Cases", "Deaths"]] = scaler.fit_transform(scaled_data[["Cases", "Deaths"]])
# scaled_data.head()
# weekly_data_dict_scaled = time_series_to_dict(scaled_data)
#thresholds_weekly_data_scaled = get_thresholds(scaled_data)
print(weekly_data_dict)

10
13
20
21
9
{0: [13, 7, 13, 14, 13, 10, 5, 4, 3, 2, 2, 0, 4, 12, 13, 14, 5, 3, 10, 9, 14, 10, 1, 2, 4, 7, 26, 4, 12, 12, 23, 22, 27, 10, 12, 17, 44, 3, 4, 19, 7, 16, 9, 42, 57, 14, 14, 18, 17, 16, 18, 23, 4, 19, 5, 2, 18, 8, 6, 6, 3, 5, 4, 4, 4, 4, 9, 21, 19, 5, 19, 16, 33, 11, 11, 11, 66, 11, 15, 5, 13, 14, 4, 4, 3, 1, 3, 1, 3, 2, 4, 1, 0, 21, 20, 20, 20, 39, 12, 11, 5, 4, 22, 13, 13, 13, 15, 8, 11, 8, 12, 16, 16, 7, 4, 5, 4, 5, 5, 6, 2, 3, 7, 14, 16, 8, 8, 14, 8, 10, 11, 9, 150, 9, 7, 10, 14, 12, 166, 6, 5, 11, 1, 5, 6, 7, 4, 1, 2, 2, 1, 6, 0, 16, 14, 9, 19, 11, 8, 9, 21, 33, 31, 30, 65, 73, 45, 64, 61, 90, 66, 51, 26, 15, 22, 20, 12, 7, 7, 10, 8, 8, 99, 59, 55, 43, 21, 14, 21, 26, 25, 26, 26, 38, 38, 35, 24, 15, 14, 18, 11, 12, 11, 8, 17, 16, 21, 12, 8, 20, 47, 66, 51, 67, 46, 48, 11, 14, 12, 18, 15, 25, 20, 17, 24, 15, 20, 16, 17, 11, 9, 16, 15, 12, 14, 17, 20, 18, 16, 14, 17, 14, 65, 63, 37, 57, 44, 28, 9, 5, 7, 5, 8, 5, 6, 5, 11, 10, 9, 6, 7, 10, 11, 10, 8, 0, 5, 6, 9, 7, 12, 1

In [203]:
#As a quick note - all of the LSTM tests were done on the old threshold data - have not corrected to see if the 
#results changed but I suspect there would be no difference
test = []
burkina_faso_data = []
def return_features_labels(window_size, weekly_dict, thresholds, state=True):
    #input data is an array of tuples (features, label) where features is the sequence data of no. of cases and label 
#is the state
    input_data = []
    #array of tuples of (features, label) where label is also a tuple of (country, no. of case at t+1, state at t+1)
    continuous_data = []
    for country, cases in weekly_dict.items():
        for i in range(len(cases)-window_size-1):
            features = cases[i:i+window_size]
            label = find_state(thresholds[country], cases[i+1+window_size])
            input_data.append((features, label))
            continuous_data.append((features, (country, cases[i+1+window_size], label)))
            if country == 1 and window_size==4:
                burkina_faso_data.append((features, (country, cases[i+1+window_size], label)))
    if state == False:
        return continuous_data
    return input_data
#uncommented to so burkina_faso data won't work
# a = return_features_labels(4, weekly_data_dict, thresholds_weekly_data)
input_data_4 = return_features_labels(4, weekly_data_dict, thresholds_weekly_data)
input_data_5 = return_features_labels(5, weekly_data_dict, thresholds_weekly_data, state=False)
input_data_6 = return_features_labels(6, weekly_data_dict, thresholds_weekly_data, state=False)
input_data_7 = return_features_labels(7, weekly_data_dict, thresholds_weekly_data, state=False)
continuous_data = return_features_labels(4, weekly_data_dict, thresholds_weekly_data, state = False)
continuous_data_10 = return_features_labels(10, weekly_data_dict, thresholds_weekly_data, state=False)
print(continuous_data_10)
# input_data_4_scaled = return_features_labels(4, weekly_data_dict_scaled, thresholds_weekly_data_scaled)

[([13, 7, 13, 14, 13, 10, 5, 4, 3, 2], (0, 0, 0)), ([7, 13, 14, 13, 10, 5, 4, 3, 2, 2], (0, 4, 0)), ([13, 14, 13, 10, 5, 4, 3, 2, 2, 0], (0, 12, 1)), ([14, 13, 10, 5, 4, 3, 2, 2, 0, 4], (0, 13, 1)), ([13, 10, 5, 4, 3, 2, 2, 0, 4, 12], (0, 14, 1)), ([10, 5, 4, 3, 2, 2, 0, 4, 12, 13], (0, 5, 0)), ([5, 4, 3, 2, 2, 0, 4, 12, 13, 14], (0, 3, 0)), ([4, 3, 2, 2, 0, 4, 12, 13, 14, 5], (0, 10, 1)), ([3, 2, 2, 0, 4, 12, 13, 14, 5, 3], (0, 9, 1)), ([2, 2, 0, 4, 12, 13, 14, 5, 3, 10], (0, 14, 1)), ([2, 0, 4, 12, 13, 14, 5, 3, 10, 9], (0, 10, 1)), ([0, 4, 12, 13, 14, 5, 3, 10, 9, 14], (0, 1, 0)), ([4, 12, 13, 14, 5, 3, 10, 9, 14, 10], (0, 2, 0)), ([12, 13, 14, 5, 3, 10, 9, 14, 10, 1], (0, 4, 0)), ([13, 14, 5, 3, 10, 9, 14, 10, 1, 2], (0, 7, 0)), ([14, 5, 3, 10, 9, 14, 10, 1, 2, 4], (0, 26, 2)), ([5, 3, 10, 9, 14, 10, 1, 2, 4, 7], (0, 4, 0)), ([3, 10, 9, 14, 10, 1, 2, 4, 7, 26], (0, 12, 1)), ([10, 9, 14, 10, 1, 2, 4, 7, 26, 4], (0, 12, 1)), ([9, 14, 10, 1, 2, 4, 7, 26, 4, 12], (0, 23, 2)), ([14, 10,

In [204]:
for country, values in weekly_data_dict.items():
    print(country_list[country], " : ", len(values))

Benin  :  399
Burkina Faso  :  425
Ivory Coast  :  335
Mali  :  403
Niger  :  380
Dem. Rep. Congo  :  195
Chad  :  303
Togo  :  366
Ghana  :  264
Nigeria  :  345
Ethiopia  :  102
Centrafrique  :  212
Guinea  :  190
Cameroun  :  168
Sudan  :  175
Senegal  :  221
South Sudan  :  100
Kenya  :  34
Uganda  :  19
Burundi  :  22


Above is a list of all of the countries being included in the final input_data that is passed into the LSTM. The number of recorded datapoints for different countries seems to be extremely disproportionate. This means that once source of error when testing is that the data comes from a different distribution to the training data (i.e. we train on data for example only on Burkina Faso and the test data contains data for Uganda).

One potential problem:

When I collect data, and put into a large array [1, 0, 2] I assume that each datapoint is the recorded number of cases for the week after the first one. In practice, this may not necessarily be the case for all the datapoints (could be months or even years apart) - how do I handle this?

Also difference in temporal time frame between inital weekly and later monthly data (sequence are representing different temporal timeframes)

Strategies for handling missing data
1. Only use data without gaps (as I have done)
2. Fill in the gaps (e.g. with the last observed value or the median)


### Monthly Data
We are gonna test the LSTM on monthly data but we are going to do a couple of things differently this time
1. We are gonna impute missing values by plotting a regression curve with the points we have. This will ensure the temporal difference between points is constant (which would otherwise increase the amount of noise in an already noisy dataset)

In [234]:
new_monthly_data = pd.read_csv("../Current_Data/Month_data.csv", usecols=['Country','Date','Cases','Deaths','Summary'],  
parse_dates=["Date"])
monthly_thresholds_all = get_thresholds(new_monthly_data)
for country, thresholds in monthly_thresholds_all.items():
    #*NOTE updated thresholds which are now ignored to include 0 and 1 or 0 and 2
    if (thresholds[0] == 0 and thresholds[1] == 0) or (thresholds[0] == 0 and thresholds[1] <= 2):
        indexNames = new_monthly_data[new_monthly_data['Country'] == country ].index
        new_monthly_data = new_monthly_data.drop(indexNames, axis=0)
#remove any data from 2005 since there are only 4 months
new_monthly_data = new_monthly_data.loc[new_monthly_data["Date"].dt.year != 2005]
new_monthly_data = new_monthly_data.set_index("Date", drop=True)
all_dates = list(pd.date_range("2006-1-31", periods=12*14, freq="M").values)
print(monthly_thresholds_all.keys())

dict_keys([0, 1, 5, 12, 14, 16, 22, 23, 7, 15, 6, 4, 8, 3, 19, 10, 17, 13, 18, 11, 20, 21, 9, 24, 2])


In [235]:
month_data = pd.DataFrame(columns=["Date", "Country", "Cases"])
#impute missing values
#start with these countries to see if it will work 
for country in [1,12,14,22,5,16,23,0]:
    country_df = new_monthly_data.loc[new_monthly_data["Country"] == country]
    missing_dates = []
    #resample so every date becomes last possible day of its month
    country_df = country_df.resample("M").sum()
    country_df = country_df.reset_index()
    #The code below creates a dataframe for each country that contains its initial datapoints and adds the dates
    #with missing datapoints as nan. This is to ensure all dataframes have the same size since we will be imputing
    #missing values
    filled_in_df = pd.DataFrame(columns=['Date', 'Cases'])
    for j in range(len(all_dates)):
        date = all_dates[j]
        if date not in country_df["Date"].values:
            filled_in_df.loc[j] = [date] + [np.NaN]
            missing_dates.append(date)
        else:
            filled_in_df.loc[j] = [date] + [float(country_df.loc[country_df.Date == date]["Cases"].values[0])]
    
     #convert dates to ordinal so regression can work more effectively
    country_df["Date"].map(datetime.datetime.toordinal)
    #impute missing values by plotting regression curve and finding missing values from curve
    if len(country_df) != len(all_dates):
        model = KernelRidge()
        model.fit(country_df["Date"].values.reshape(-1, 1), country_df["Cases"].values.reshape(-1, 1))
        for date in missing_dates:
            ordinal_date = datetime.datetime.toordinal(date)
            input_prediction = np.array(ordinal_date).reshape(1, -1)
            imputed_calculation = int(model.predict(input_prediction)[0][0])
            filled_in_df.loc[filled_in_df["Date"] == date, "Cases"] = imputed_calculation
    filled_in_df["Country"] = country
#     sum_cases = filled_in_df["Cases"].sum()
#     mean = filled_in_df["Cases"].mean()
#     std = filled_in_df["Cases"].std()
#     filled_in_df["Cases"] = filled_in_df["Cases"].apply(lambda x: (x-mean)/std)
    month_data = pd.concat((month_data, filled_in_df))

monthly_thresholds = get_thresholds(month_data)
month_data["State"] = month_data.apply(lambda x: find_state(monthly_thresholds[x["Country"]], x["Cases"]) ,axis=1)
new_monthly_data_dict = time_series_to_dict(month_data)
month_data.to_csv("updated_monthly_data.csv", index=False, header=False)


### Prepare Monthly Data for Natural Language Processing Task

In [236]:

new_monthly_data["State"] = new_monthly_data.apply(lambda x: find_state(monthly_thresholds_all[x["Country"]], 
                                                   x["Cases"]), axis=1)
new_monthly_data = new_monthly_data.reset_index()
#Countries for which there is available monthly data => label will be array of length 20
countries_monthly = np.array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 14, 15, 16, 17, 18,
       22, 23, 24])

#this function will take a dataframe that contains either weekly or monthly datapoints and convert into a 
#dataframe that can be used by a language model like BERT/Longformer for multilabel classification
def prepare_df_to_lang(df):
    #This dataframe will contains individual samples of the summary with just a summary and the combined 
    #representative state - state represents the sum of the states of the countries in each report while count 
    #represents the number of countries
    summary_state = pd.DataFrame(columns=["Summary", "State", "Count", "Label"])
    #prepare multilabel classification
    multi_label_classification = pd.DataFrame(columns=["text", "labels"])
    state_total = 0
    state_count = 0
    summary = df.iloc[0]["Summary"]
    total = len(summary.split())
    curr_date = df.iloc[0]["Date"]
    max_length = 0
    #array of length 20 that will contain the state for each country at each specific index e.g. 0 benin 1 bukinra
    #would be [0, 1 etc..]
    curr_label = np.zeros(20)
    for index, row in df.iterrows():
        if row["Date"] != curr_date:
            #saved current data before resetting it
            summary_state.loc[index] = [summary] + [state_total] + [state_count] + [round(state_total/state_count)]
            multi_label_classification.loc[index] = [summary] + [curr_label]
            #reset the data
            curr_label = np.zeros(20)
            state_count = 1
            state_total = row["State"]
            summary = row["Summary"]
            curr_date = row["Date"]
            total += len(summary.split())
        else:
            state_count += 1
            state_total += row["State"]
            max_length = len(summary.split()) if len(summary.split()) > max_length else max_length
            #access correct index of curr_label by checking corresponding index from countries_monthly
            curr_index = np.where(countries_monthly==row["Country"])
            curr_label[curr_index] = 1 if row["State"] == 1 or row["State"] == 2 else 0
    return (summary_state, multi_label_classification)

_, multi_label_classification = prepare_df_to_lang(new_monthly_data)
multi_label_classification.to_csv("Text Information Extraction/multilabel_classification.csv", index=False)
multi_label_classification

,text,labels
8,The districts of Guidan-Roumji in Niger and Do...,"[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
17,The week 6 was marked by a sudden apparition o...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
26,Spatial analysis shows an epidemic area common...,"[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ..."
35,During week 14 it is noted that: Burkina Faso:...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ..."
44,Burkina Faso: Titao was the last district in a...,"[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
...,...,...
2046,"At week 23 of year 2019, no district reached t...","[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, ..."
2064,"From week 27 to 30 (July 2019), no district re...","[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, ..."
2082,"From week 31 to 35 (August 2019), no district ...","[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, ..."
2097,"From week 36 to 39 (September 2019), of the co...","[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ..."


In [237]:
#Preparing weekly data to see if there will be a difference in the multilabel classification accuracy
weekly_data_lang = data.loc[data["Country"] == 1]
weekly_data_lang = weekly_data_lang.drop(columns=["Bordering Countries", "Cases", "Deaths", "Country"])
_, multi_label_classification_weekly = prepare_df_to_lang(data)
multi_label_classification_weekly.to_csv("Text Information Extraction/multilabel_classification_weekly.csv", index=False)
weekly_data_lang.to_csv("Text Information Extraction/burkina.csv", index=False)

In [238]:
print(multi_label_classification_weekly.iloc[3]["text"])
data.iloc[24]["Summary"]

District of P, Burkina Faso reached the alert threshold at week 4, with 8 cases (attack rate 5.3 cases/100 000 inhabitants). The 5 cases come from 5 different health centres. The laboratory analysis of CSFs collected showed 3 Gram positive diplococcus; 3 Gram negative diplococcus and 2 negative results.


'District of Pô, Burkina Faso reached the alert threshold at week 4, with 8 cases (attack rate 5.3 cases/100 000 inhabitants). The 5 cases come from 5 different health centres. The laboratory analysis of CSFs collected showed 3 Gram positive diplococcus; 3 Gram negative diplococcus and 2 negative results.'